# GSM8K Genetic Algorithm for Prompt Evolution

## Complete Tutorial: Evolving Mathematical Reasoning Prompts with Async Batch Processing

This notebook provides a comprehensive tutorial for using genetic algorithms to evolve prompts for mathematical reasoning on the GSM8K dataset. **Now featuring the new asynchronous batch evaluation system for 3-8x performance improvements!**

You'll learn how to:

- Set up the system and configure async batch processing
- Run evolution experiments with high-performance concurrent evaluation
- Monitor real-time performance metrics and throughput
- Analyze results and interpret evolved prompts
- Optimize batch sizes and concurrency for your use case
- Compare async vs sync performance

**🚀 New Features:**
- **Asynchronous Batch Processing**: 3-8x faster evaluation through concurrent API calls
- **Intelligent Rate Limiting**: Automatic compliance with OpenAI API limits
- **Performance Monitoring**: Real-time throughput and efficiency metrics
- **Configurable Concurrency**: Tune batch sizes and concurrent requests for optimal performance

**Prerequisites:**
- OpenAI API key (for GPT models)
- Anthropic API key (for Claude models) - optional
- Python environment with required dependencies

---

## 1. System Setup and Dependencies

First, let's set up the environment and import all necessary modules.

In [1]:
# Install required packages if not already installed
import subprocess
import sys

def install_package(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# Uncomment and run if packages are not installed
# install_package("openai>=1.0.0")
# install_package("anthropic")
# install_package("matplotlib")
# install_package("numpy")
# install_package("psutil")
# install_package("aiohttp")  # Required for async batch processing
# install_package("asyncio")  # Required for async operations

print("✅ Dependencies ready (including async batch processing support)")

✅ Dependencies ready (including async batch processing support)


In [2]:
# Import system modules
import os
import sys
import time
from pathlib import Path

# Add project root to Python path
project_root = Path.cwd()
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

print(f"📁 Project root: {project_root}")
print("✅ System imports ready")

📁 Project root: /Users/Odyssey/Projects/genetic-prompt
✅ System imports ready


## 2. API Configuration

Configure your API keys for accessing language models. The system supports both OpenAI and Anthropic models.

In [3]:
# Set up API keys
# Option 1: Set environment variables (recommended)
# os.environ["OPENAI_API_KEY"] = "your-openai-api-key-here"
# os.environ["ANTHROPIC_API_KEY"] = "your-anthropic-api-key-here"

# Option 2: Load from .env file
env_file = project_root / ".env"
if env_file.exists():
    with open(env_file, 'r') as f:
        for line in f:
            if '=' in line and not line.startswith('#'):
                key, value = line.strip().split('=', 1)
                os.environ[key] = value
    print("✅ Environment variables loaded from .env file")
else:
    print("⚠️  No .env file found. Please set API keys manually.")

# Verify API keys are set
openai_key = os.getenv("OPENAI_API_KEY")
anthropic_key = os.getenv("ANTHROPIC_API_KEY")

print(f"🔑 OpenAI API Key: {'✅ Set' if openai_key else '❌ Not set'}")
print(f"🔑 Anthropic API Key: {'✅ Set' if anthropic_key else '❌ Not set'}")

✅ Environment variables loaded from .env file
🔑 OpenAI API Key: ✅ Set
🔑 Anthropic API Key: ✅ Set


## 3. Load System Components

Now let's load all the genetic algorithm components, including the new asynchronous batch evaluation system.

In [4]:
# Import genetic algorithm components
import asyncio
from src.utils.config import config
from src.embeddings.vocabulary import vocabulary
from src.seeds.seed_manager import SeedManager
from src.config.experiment_configs import ConfigurationManager

# Import new async batch evaluation components
from src.genetics.async_evolution import AsyncEvolutionController, AsyncEvolutionConfig
from src.evaluation.async_pipeline import AsyncEvaluationPipeline, PopulationBatchConfig
from src.evaluation.async_llm_interface import AsyncLLMInterface, BatchConfig

print("✅ Core components imported (including async batch evaluation system)")

Cache loaded: 0 evaluation entries, 0 fitness entries
✅ Core components imported (including async batch evaluation system)


In [5]:
# Initialize vocabulary
vocab_file = config.get_data_dir() / "embeddings" / "vocabulary.pkl"

if vocab_file.exists():
    vocabulary.load_vocabulary(vocab_file)
    print(f"✅ Vocabulary loaded: {len(vocabulary.token_to_id)} tokens")
else:
    print("📚 Creating vocabulary from scratch...")
    vocabulary._create_basic_vocabulary()
    print(f"✅ Basic vocabulary created: {len(vocabulary.token_to_id)} tokens")

Vocabulary loaded from /Users/Odyssey/Projects/genetic-prompt/data/embeddings/vocabulary.pkl
Vocabulary size: 10004
✅ Vocabulary loaded: 10004 tokens


In [6]:
# Initialize seed manager and configuration manager
seed_manager = SeedManager()
config_manager = ConfigurationManager()

# Load base seed collection
base_seeds = seed_manager.get_base_seeds()
print(f"🌱 Seed collection loaded: {len(base_seeds)} high-quality prompts")

# Show available experiment presets
presets = config_manager.list_presets()
print(f"⚙️  Available presets: {', '.join(presets)}")

📂 Loaded collection 'test_collection' with 10 seeds
📂 Loaded collection 'test_validation_collection' with 10 seeds
🌱 Seed collection loaded: 50 high-quality prompts
⚙️  Available presets: quick_test, standard, thorough, ablation_no_crossover, ablation_no_mutation, high_mutation, large_population, random_search


## 4. Explore Seed Prompts

Let's examine the high-quality seed prompts that will initialize our genetic algorithm.

In [7]:
# Show seed prompt categories and examples
from src.seeds.prompt_categories import PromptCategory

print("📂 Seed Prompt Categories:")
print("=" * 40)

for category in PromptCategory:
    category_seeds = seed_manager.get_seeds_by_category(category)
    print(f"\n🔹 {category.value.replace('_', ' ').title()}: {len(category_seeds)} prompts")
    
    # Show first example
    if category_seeds:
        example = category_seeds[0]
        print(f"   Example: \"{example.text}\"")
        print(f"   Strength: {example.expected_strength}")

📂 Seed Prompt Categories:

🔹 Step By Step: 8 prompts
   Example: "Let's solve this step by step."
   Strength: Clear sequential reasoning

🔹 Visual Reasoning: 4 prompts
   Example: "Let me visualize this problem to better understand it."
   Strength: Better spatial understanding

🔹 Algebraic Approach: 5 prompts
   Example: "Let me define variables and set up equations for this problem."
   Strength: Handles unknown quantities

🔹 Logical Breakdown: 5 prompts
   Example: "Let me think about this logically and reason through each part."
   Strength: Clear reasoning chains

🔹 Pattern Recognition: 4 prompts
   Example: "I notice a pattern here that can help solve this more efficiently."
   Strength: Efficient solutions

🔹 Estimation Checking: 5 prompts
   Example: "Let me estimate the answer first, then calculate precisely."
   Strength: Error detection through estimation

🔹 Word Problem Parsing: 6 prompts
   Example: "Let me carefully read and understand what this problem is asking."
   St

In [8]:
# Validate seed collection quality
from src.seeds.seed_validation import SeedValidator

validator = SeedValidator()
validation_metrics = validator.validate_collection(base_seeds)

print("🔍 Seed Collection Quality Report:")
print("=" * 40)
print(f"Overall Score: {validation_metrics.overall_score:.3f}")
print(f"Diversity Score: {validation_metrics.diversity_score:.3f}")
print(f"Category Balance: {validation_metrics.category_balance:.3f}")
print(f"Uniqueness Score: {validation_metrics.uniqueness_score:.3f}")

quality_status = "🟢 EXCELLENT" if validation_metrics.overall_score >= 0.8 else "🟡 GOOD" if validation_metrics.overall_score >= 0.6 else "🔴 NEEDS IMPROVEMENT"
print(f"\nQuality Status: {quality_status}")

🔍 Seed Collection Quality Report:
Overall Score: 0.898
Diversity Score: 0.771
Category Balance: 1.000
Uniqueness Score: 0.912

Quality Status: 🟢 EXCELLENT


## 5. Interactive Hyperparameter Configuration

Use the interactive interface below to configure all genetic algorithm hyperparameters with real-time validation and visual feedback.

In [9]:
# Show available experiment presets
preset_info = config_manager.get_preset_info()

print("⚙️  Available Experiment Presets:")
print("=" * 50)

for name, info in preset_info.items():
    print(f"\n🔹 {name}")
    print(f"   Name: {info['name']}")
    print(f"   Description: {info['description']}")
    print(f"   Population: {info['population_size']}, Generations: {info['max_generations']}")
    print(f"   Problems: {info['max_problems']}")

⚙️  Available Experiment Presets:

🔹 quick_test
   Name: Quick Test
   Description: Fast test run for system validation
   Population: 10, Generations: 15
   Problems: 20

🔹 standard
   Name: Standard Evolution
   Description: Standard GSM8K evolution experiment
   Population: 50, Generations: 100
   Problems: 100

🔹 thorough
   Name: Thorough Evolution
   Description: Comprehensive evolution with large population
   Population: 100, Generations: 200
   Problems: 200

🔹 ablation_no_crossover
   Name: Ablation: No Crossover
   Description: Evolution with mutation only (no crossover)
   Population: 50, Generations: 100
   Problems: 100

🔹 ablation_no_mutation
   Name: Ablation: No Mutation
   Description: Evolution with crossover only (no mutation)
   Population: 50, Generations: 100
   Problems: 100

🔹 high_mutation
   Name: High Mutation Rate
   Description: Evolution with high mutation rate
   Population: 50, Generations: 100
   Problems: 100

🔹 large_population
   Name: Large Populat

In [10]:
# Interactive Hyperparameter Configuration Interface
from src.config.notebook_interface import display_hyperparameter_interface, quick_config_panel
from src.config.hyperparameters import get_hyperparameter_config

print("🎛️ Interactive Hyperparameter Configuration Interface")
print("=" * 60)
print("Use the interface below to configure all genetic algorithm parameters:")
print("- Adjust sliders and checkboxes to modify parameters")
print("- View parameter descriptions and valid ranges")
print("- Load presets or save custom configurations")
print("- Apply changes with real-time validation")
print()

# Display the full interactive interface
interface = display_hyperparameter_interface()
display(interface)

🎛️ Interactive Hyperparameter Configuration Interface
Use the interface below to configure all genetic algorithm parameters:
- Adjust sliders and checkboxes to modify parameters
- View parameter descriptions and valid ranges
- Load presets or save custom configurations
- Apply changes with real-time validation

Creating hyperparameter interface...


In [11]:
# Quick Configuration Panel (Alternative)
# Use this for quick adjustments to the most common parameters

print("⚡ Quick Configuration Panel")
print("=" * 40)
print("Adjust the most commonly used parameters:")
print()

quick_panel = quick_config_panel()
display(quick_panel)

⚡ Quick Configuration Panel
Adjust the most commonly used parameters:



In [12]:
# 🔧 UNIFIED CONFIGURATION - Single Source of Truth
# This section defines all key parameters to eliminate conflicts
STANDARD_CONFIG = {
    # Core Evolution Parameters
    'population_size': 50,
    'max_generations': 100,
    'max_problems': 100,
    
    # Async Batch Processing (Conservative for stability)
    'async_batch_size': 20,
    'max_concurrent_requests': 5,  # Reduced for rate limit safety
    'genome_batch_size': 10,
    'max_concurrent_genomes': 3,
    'rate_limit_per_minute': 1000,  # Conservative rate limiting
    
    # Model Configuration
    'model_name': 'gpt-4o',
    'temperature': 0.0,  # Deterministic for consistent results
    'target_fitness': 0.85,
}

print("🔧 Unified Configuration Loaded:")
print("=" * 40)
for key, value in STANDARD_CONFIG.items():
    print(f"   {key}: {value}")
print("\n✅ All configuration sections will use these values")
print()

# Choose and customize your experiment configuration
# Options: 'quick_test', 'standard', 'thorough', 'high_mutation', 'large_population', etc.

BASE_PRESET = "quick_test"  # Change this to your preferred preset

# Custom modifications (using STANDARD_CONFIG values)
custom_modifications = {
    'name': 'My GSM8K Evolution Experiment',
    'description': 'Custom experiment for prompt evolution',
    'population_size': STANDARD_CONFIG['population_size'],
    'max_generations': STANDARD_CONFIG['max_generations'],
    'max_problems': STANDARD_CONFIG['max_problems'],
    'model_name': STANDARD_CONFIG['model_name'],
    'temperature': STANDARD_CONFIG['temperature'],
    'target_fitness': STANDARD_CONFIG['target_fitness'],
}

# Create the configuration
experiment_config = config_manager.create_custom_config(BASE_PRESET, custom_modifications)

# Show the final configuration
print("🔧 Experiment Configuration:")
print("=" * 40)
print(config_manager.get_config_summary(experiment_config))

🔧 Unified Configuration Loaded:
   population_size: 50
   max_generations: 100
   max_problems: 100
   async_batch_size: 20
   max_concurrent_requests: 5
   genome_batch_size: 10
   max_concurrent_genomes: 3
   rate_limit_per_minute: 1000
   model_name: gpt-4o
   temperature: 0.0
   target_fitness: 0.85

✅ All configuration sections will use these values

🔧 Experiment Configuration:
📋 My GSM8K Evolution Experiment
   Custom experiment for prompt evolution
   Type: quick_test
   Population: 50
   Generations: 100
   Problems: 100
   Crossover: 80.0%
   Mutation: 20.0%
   Selection: tournament
   Model: gpt-4o
   Target Fitness: 0.85


In [13]:
# Show how hyperparameters are now centralized
from src.config.hyperparameters import get_hyperparameter_config

print("🎯 Centralized Hyperparameter Configuration")
print("=" * 50)
print("All genetic algorithm parameters are now centrally managed:")
print()

hyperparams = get_hyperparameter_config()

# Show key parameters
print(f"📊 Evolution Parameters:")
print(f"   Population Size: {hyperparams.population_size}")
print(f"   Max Generations: {hyperparams.max_generations}")
print(f"   Crossover Rate: {hyperparams.crossover_rate}")
print(f"   Mutation Rate: {hyperparams.mutation_rate}")
print(f"   Elite Size: {hyperparams.elite_size}")
print(f"   Tournament Size: {hyperparams.tournament_size}")
print()

print(f"🎯 Convergence Parameters:")
print(f"   Target Fitness: {hyperparams.target_fitness}")
print(f"   Convergence Patience: {hyperparams.convergence_patience}")
print(f"   Diversity Threshold: {hyperparams.diversity_threshold}")
print()

print(f"🧬 Mutation Parameters:")
print(f"   Semantic Probability: {hyperparams.semantic_prob}")
print(f"   Insertion Rate: {hyperparams.insertion_rate}")
print(f"   Deletion Rate: {hyperparams.deletion_rate}")
print(f"   Max Genome Length: {hyperparams.max_genome_length}")
print()

print(f"📝 Evaluation Parameters:")
print(f"   Max Problems: {hyperparams.max_problems}")
print(f"   Batch Size: {hyperparams.batch_size}")
print(f"   API Timeout: {hyperparams.api_timeout}s")
print(f"   Use Cache: {hyperparams.use_cache}")
print()

print("✨ Benefits of Centralized Configuration:")
print("   • All parameters in one place with validation")
print("   • Interactive notebook interface for easy modification")
print("   • Preset configurations for different experiment types")
print("   • Real-time parameter validation and error checking")
print("   • Consistent parameter usage across all modules")
print("   • New async batch processing parameters for performance optimization")

🎯 Centralized Hyperparameter Configuration
All genetic algorithm parameters are now centrally managed:

📊 Evolution Parameters:
   Population Size: 50
   Max Generations: 100
   Crossover Rate: 0.8
   Mutation Rate: 0.2
   Elite Size: 5
   Tournament Size: 3

🎯 Convergence Parameters:
   Target Fitness: 0.85
   Convergence Patience: 20
   Diversity Threshold: 0.05

🧬 Mutation Parameters:
   Semantic Probability: 0.9
   Insertion Rate: 0.05
   Deletion Rate: 0.05
   Max Genome Length: 50

📝 Evaluation Parameters:
   Max Problems: 100
   Batch Size: 10
   API Timeout: 30s
   Use Cache: True

✨ Benefits of Centralized Configuration:
   • All parameters in one place with validation
   • Interactive notebook interface for easy modification
   • Preset configurations for different experiment types
   • Real-time parameter validation and error checking
   • Consistent parameter usage across all modules
   • New async batch processing parameters for performance optimization


In [14]:
# Validate the configuration
validation_errors = config_manager.validate_config(experiment_config)

if validation_errors:
    print("❌ Configuration validation failed:")
    for error in validation_errors:
        print(f"   - {error}")
else:
    print("✅ Configuration is valid and ready to use!")

✅ Configuration is valid and ready to use!


## 5.5. Configure Asynchronous Batch Processing

Configure the new async batch evaluation system for optimal performance. This system provides 3-8x speedup over sequential evaluation.

In [15]:
# Configure async batch processing parameters
from src.config.hyperparameters import get_hyperparameter_config

print("🚀 Asynchronous Batch Processing Configuration")
print("=" * 60)

hyperparams = get_hyperparameter_config()

print(f"📊 Current Async Settings:")
print(f"   Enable Async Evaluation: {hyperparams.enable_async_evaluation}")
print(f"   Async Batch Size: {hyperparams.async_batch_size} problems/batch")
print(f"   Max Concurrent Requests: {hyperparams.max_concurrent_requests}")
print(f"   Genome Batch Size: {hyperparams.genome_batch_size} genomes/batch")
print(f"   Max Concurrent Genomes: {hyperparams.max_concurrent_genomes}")
print(f"   Rate Limit: {hyperparams.rate_limit_per_minute} requests/minute")
print()

# Show configuration recommendations
print("⚙️  Configuration Recommendations:")
print()
print("🟢 Conservative (Rate Limit Safe):")
print("   async_batch_size=10, max_concurrent_requests=5")
print("   genome_batch_size=5, max_concurrent_genomes=3")
print("   Expected speedup: 2-3x")
print()
print("🟡 Balanced (Recommended):")
print("   async_batch_size=20, max_concurrent_requests=10")
print("   genome_batch_size=10, max_concurrent_genomes=5")
print("   Expected speedup: 3-5x")
print()
print("🔴 Aggressive (Maximum Performance):")
print("   async_batch_size=30, max_concurrent_requests=15")
print("   genome_batch_size=15, max_concurrent_genomes=8")
print("   Expected speedup: 5-8x (monitor rate limits)")

🚀 Asynchronous Batch Processing Configuration
📊 Current Async Settings:
   Enable Async Evaluation: True
   Async Batch Size: 20 problems/batch
   Max Concurrent Requests: 10
   Genome Batch Size: 10 genomes/batch
   Max Concurrent Genomes: 5
   Rate Limit: 3500 requests/minute

⚙️  Configuration Recommendations:

🟢 Conservative (Rate Limit Safe):
   async_batch_size=10, max_concurrent_requests=5
   genome_batch_size=5, max_concurrent_genomes=3
   Expected speedup: 2-3x

🟡 Balanced (Recommended):
   async_batch_size=20, max_concurrent_requests=10
   genome_batch_size=10, max_concurrent_genomes=5
   Expected speedup: 3-5x

🔴 Aggressive (Maximum Performance):
   async_batch_size=30, max_concurrent_requests=15
   genome_batch_size=15, max_concurrent_genomes=8
   Expected speedup: 5-8x (monitor rate limits)


In [16]:
# Create async evolution configuration
async_config = AsyncEvolutionConfig(
    # Basic evolution parameters
    population_size=experiment_config.population_size,
    max_generations=experiment_config.max_generations,
    crossover_rate=experiment_config.crossover_rate,
    mutation_rate=experiment_config.mutation_rate,
    elite_size=experiment_config.elite_size,
    target_fitness=experiment_config.target_fitness,
    
    # Async batch processing settings (using STANDARD_CONFIG)
    enable_async_evaluation=True,
    async_batch_size=STANDARD_CONFIG['async_batch_size'],
    max_concurrent_requests=STANDARD_CONFIG['max_concurrent_requests'],
    genome_batch_size=STANDARD_CONFIG['genome_batch_size'],
    max_concurrent_genomes=STANDARD_CONFIG['max_concurrent_genomes'],
    rate_limit_per_minute=STANDARD_CONFIG['rate_limit_per_minute'],
    
    # Performance monitoring
    detailed_performance_logging=True
)

print("🔧 Async Evolution Configuration Created:")
print("=" * 50)
print(f"Population Size: {async_config.population_size}")
print(f"Max Generations: {async_config.max_generations}")
print(f"Async Batch Size: {async_config.async_batch_size}")
print(f"Concurrent Requests: {async_config.max_concurrent_requests}")
print(f"Genome Batch Size: {async_config.genome_batch_size}")
print(f"Concurrent Genomes: {async_config.max_concurrent_genomes}")
print()
print(f"📈 Expected Performance:")
total_problems = async_config.population_size * experiment_config.max_problems
print(f"   Total API calls per generation: ~{total_problems}")
print(f"   Expected speedup: 3-5x over sequential processing")
print(f"   Estimated throughput: 15-25 problems/second")

# Configuration validation
print("\n🔍 Configuration Validation:")
print("=" * 40)
config_valid = True

# Check for consistency
if async_config.population_size != STANDARD_CONFIG['population_size']:
    print(f"⚠️  Population size mismatch: {async_config.population_size} vs {STANDARD_CONFIG['population_size']}")
    config_valid = False

if async_config.max_generations != STANDARD_CONFIG['max_generations']:
    print(f"⚠️  Max generations mismatch: {async_config.max_generations} vs {STANDARD_CONFIG['max_generations']}")
    config_valid = False

if async_config.async_batch_size != STANDARD_CONFIG['async_batch_size']:
    print(f"⚠️  Async batch size mismatch: {async_config.async_batch_size} vs {STANDARD_CONFIG['async_batch_size']}")
    config_valid = False

if config_valid:
    print("✅ All configurations are consistent with STANDARD_CONFIG")
else:
    print("❌ Configuration inconsistencies detected - please review")

🔧 Async Evolution Configuration Created:
Population Size: 50
Max Generations: 100
Async Batch Size: 20
Concurrent Requests: 5
Genome Batch Size: 10
Concurrent Genomes: 3

📈 Expected Performance:
   Total API calls per generation: ~5000
   Expected speedup: 3-5x over sequential processing
   Estimated throughput: 15-25 problems/second

🔍 Configuration Validation:
✅ All configurations are consistent with STANDARD_CONFIG


## 6. Set Up Monitoring and Visualization

Before running the experiment, let's set up real-time monitoring and visualization with enhanced async performance tracking.

In [17]:
# Import monitoring components
from src.utils.experiment_manager import ExperimentManager
from src.utils.evolution_logging import EvolutionLogger
from src.utils.visualization import EvolutionVisualizer
from src.utils.performance_monitor import PerformanceMonitor

# Initialize experiment manager
experiment_manager = ExperimentManager()

print("📊 Monitoring components initialized")
print("🚀 Async performance monitoring enabled")
print("✅ Ready for high-performance async experiment execution")

📊 Monitoring components initialized
🚀 Async performance monitoring enabled
✅ Ready for high-performance async experiment execution


## 7. Run the Async Evolution Experiment

Now we'll run the complete genetic algorithm experiment using the new asynchronous batch evaluation system with real-time performance monitoring.

In [18]:
# Initialize the async evolution controller
from dataclasses import asdict

print("🚀 Initializing async evolution controller...")
print("=" * 60)
print(f"🧬 Population Size: {async_config.population_size}")
print(f"🔄 Max Generations: {async_config.max_generations}")
print(f"📊 Evaluation Problems: {experiment_config.max_problems}")
print(f"🤖 Model: {experiment_config.model_name}")
print(f"⚡ Async Batch Size: {async_config.async_batch_size}")
print(f"🔀 Concurrent Requests: {async_config.max_concurrent_requests}")
print(f"🧪 Genome Batch Size: {async_config.genome_batch_size}")
print("=" * 60)

# Create async evolution controller
# Convert SeedPrompt objects to text strings (CRITICAL FIX)
if base_seeds:
    seed_texts = [seed.text for seed in base_seeds[:10]]
    print(f"🔧 Converted {len(seed_texts)} SeedPrompt objects to text strings")
    print(f"📝 Sample seed text: '{seed_texts[0][:50]}...'")
else:
    seed_texts = None
    print("⚠️  No base_seeds available, using None")

# Validation: Ensure we're passing strings, not SeedPrompt objects
if seed_texts:
    for i, text in enumerate(seed_texts[:3]):
        if hasattr(text, 'text'):  # This would indicate a SeedPrompt object
            raise TypeError(f"ERROR: seed_texts[{i}] is still a SeedPrompt object, not a string!")
        if not isinstance(text, str):
            raise TypeError(f"ERROR: seed_texts[{i}] is {type(text)}, expected str!")
    print(f"✅ Validation passed: All {len(seed_texts)} seed_texts are strings")

async_controller = AsyncEvolutionController(
    config=async_config,
    seed_prompts=seed_texts  # Use converted text strings, NOT SeedPrompt objects
)

print("✅ Async evolution controller initialized successfully!")
print(f"📈 Expected performance improvement: 3-5x over sequential processing")

🚀 Initializing async evolution controller...
🧬 Population Size: 50
🔄 Max Generations: 100
📊 Evaluation Problems: 100
🤖 Model: gpt-4o
⚡ Async Batch Size: 20
🔀 Concurrent Requests: 5
🧪 Genome Batch Size: 10
🔧 Converted 10 SeedPrompt objects to text strings
📝 Sample seed text: 'Let's solve this step by step....'
✅ Validation passed: All 10 seed_texts are strings
Initialized population with 10 seeds and 40 random genomes
Initialized population with 10 seed prompts
✅ Population initialized with 50 genomes
✅ Async evolution controller initialized successfully!
📈 Expected performance improvement: 3-5x over sequential processing


In [19]:
# Performance comparison setup
print("🔧 Setting up performance comparison...")
print("This experiment will demonstrate the async batch processing performance improvements.")
print()

# Estimate performance improvements
total_evaluations = async_config.population_size * experiment_config.max_problems
estimated_sync_time = total_evaluations * 0.5  # ~0.5 seconds per evaluation (sequential)
estimated_async_time = estimated_sync_time / 4  # ~4x speedup expected

print(f"📊 Performance Estimates:")
print(f"   Total evaluations per generation: {total_evaluations}")
print(f"   Estimated sync time: {estimated_sync_time/60:.1f} minutes")
print(f"   Estimated async time: {estimated_async_time/60:.1f} minutes")
print(f"   Expected speedup: ~4x faster")
print()
print("✅ Ready to run async evolution experiment")

🔧 Setting up performance comparison...
This experiment will demonstrate the async batch processing performance improvements.

📊 Performance Estimates:
   Total evaluations per generation: 5000
   Estimated sync time: 41.7 minutes
   Estimated async time: 10.4 minutes
   Expected speedup: ~4x faster

✅ Ready to run async evolution experiment


In [20]:
# Run the async evolution experiment
print("🧬 Starting asynchronous genetic algorithm evolution...")
print("=" * 70)
print(f"🚀 Using Async Batch Processing System")
print(f"📊 Population Size: {async_config.population_size}")
print(f"🔄 Max Generations: {async_config.max_generations}")
print(f"📝 Evaluation Problems: {experiment_config.max_problems}")
print(f"🤖 Model: {experiment_config.model_name}")
print(f"⚡ Batch Size: {async_config.async_batch_size} problems/batch")
print(f"🔀 Concurrent Requests: {async_config.max_concurrent_requests}")
print("=" * 70)
print()

# Define async experiment function
async def run_async_experiment():
    """Run the complete async evolution experiment."""
    start_time = time.time()
    
    try:
        # Run the async evolution
        results = await async_controller.run_evolution_async(
            max_generations=async_config.max_generations
        )
        
        total_time = time.time() - start_time
        
        print(f"\n🎉 Async Evolution Completed Successfully!")
        print(f"⏱️  Total experiment time: {total_time:.1f} seconds ({total_time/60:.1f} minutes)")
        
        return results, True, total_time
        
    except Exception as e:
        total_time = time.time() - start_time
        print(f"❌ Async evolution failed: {e}")
        print(f"⏱️  Time before failure: {total_time:.1f} seconds")
        return None, False, total_time

# Run the async experiment
print("🚀 Launching async evolution...")
experiment_results, experiment_success, total_experiment_time = await run_async_experiment()

if experiment_success:
    print(f"\n📈 Performance Summary:")
    print(f"   Best Fitness: {experiment_results['best_fitness']:.3f}")
    print(f"   Total Generations: {experiment_results['total_generations']}")
    print(f"   Total Evaluations: {experiment_results['total_evaluations']}")
    print(f"   Average Evaluation Time: {experiment_results['performance_summary']['average_async_eval_time']:.2f}s per generation")
    
    # Show async performance stats
    async_stats = experiment_results.get('async_stats', {})
    if async_stats:
        pipeline_stats = async_stats.get('pipeline_stats', {})
        print(f"\n🚀 Async Performance Metrics:")
        print(f"   API Calls Made: {pipeline_stats.get('api_calls_made', 0)}")
        print(f"   Cache Hits: {pipeline_stats.get('cache_hits', 0)}")
        print(f"   Total Evaluation Time: {pipeline_stats.get('total_evaluation_time', 0):.1f}s")
        
        # Calculate throughput
        total_problems = experiment_results['total_evaluations'] * experiment_config.max_problems
        throughput = total_problems / total_experiment_time if total_experiment_time > 0 else 0
        print(f"   Throughput: {throughput:.1f} problems/second")
else:
    print("❌ Experiment failed. Please check the error messages above.")

🧬 Starting asynchronous genetic algorithm evolution...
🚀 Using Async Batch Processing System
📊 Population Size: 50
🔄 Max Generations: 100
📝 Evaluation Problems: 100
🤖 Model: gpt-4o
⚡ Batch Size: 20 problems/batch
🔀 Concurrent Requests: 5

🚀 Launching async evolution...
🧬 Starting async evolution with 50 genomes for 100 generations
📊 Async config: batch_size=20, concurrent_requests=5, genome_batch_size=10
📊 Gen 0: 50 problems
🧬 Evaluating 50 genomes in 5 batches


Evaluating population:   0%|          | 0/50 [00:00<?, ?it/s]

Processing 50 problems in 3 batches of size 20
Processing 50 problems in 3 batches of size 20
Processing 50 problems in 3 batches of size 20


Evaluating population:   0%|          | 0/50 [02:10<?, ?it/s, genome=seed_0, problem=20/50, correct=0]

Batch 1/3 completed in 130.34s (20 problems)


...

Evaluating population:   0%|          | 0/50 [15:42<?, ?it/s, genome=seed_6, problem=20/50, correct=0]

Batch 1/3 completed in 169.59s (20 problems)


Evaluating population:   0%|          | 0/50 [15:55<?, ?it/s, genome=seed_7, problem=20/50, correct=0]

Batch 1/3 completed in 169.50s (20 problems)


CancelledError: 

In [ ]:
# Performance Comparison and Benchmarking
print("📊 Async vs Sync Performance Comparison")
print("=" * 50)

if experiment_success and experiment_results:
    # Calculate performance metrics
    total_problems_processed = experiment_results['total_evaluations'] * experiment_config.max_problems
    async_throughput = total_problems_processed / total_experiment_time if total_experiment_time > 0 else 0
    
    # Estimate sync performance (based on typical sequential processing)
    estimated_sync_time = total_problems_processed * 0.5  # ~0.5 seconds per problem
    estimated_sync_throughput = total_problems_processed / estimated_sync_time if estimated_sync_time > 0 else 0
    
    speedup_factor = estimated_sync_time / total_experiment_time if total_experiment_time > 0 else 0
    
    print(f"🚀 Async Performance:")
    print(f"   Total Time: {total_experiment_time:.1f}s ({total_experiment_time/60:.1f} minutes)")
    print(f"   Throughput: {async_throughput:.1f} problems/second")
    print(f"   Problems Processed: {total_problems_processed:,}")
    print()
    print(f"🐌 Estimated Sync Performance:")
    print(f"   Estimated Time: {estimated_sync_time:.1f}s ({estimated_sync_time/60:.1f} minutes)")
    print(f"   Estimated Throughput: {estimated_sync_throughput:.1f} problems/second")
    print()
    print(f"📈 Performance Improvement:")
    print(f"   Speedup Factor: {speedup_factor:.1f}x faster")
    print(f"   Time Saved: {(estimated_sync_time - total_experiment_time)/60:.1f} minutes")
    print(f"   Efficiency Gain: {((speedup_factor - 1) * 100):.0f}% improvement")
    
    # Show batch processing efficiency
    async_stats = experiment_results.get('async_stats', {})
    if async_stats:
        batch_config = async_stats.get('batch_config', {})
        print(f"\n⚙️  Batch Processing Configuration:")
        print(f"   Batch Size: {batch_config.get('async_batch_size', 'N/A')}")
        print(f"   Max Concurrent Requests: {batch_config.get('max_concurrent_requests', 'N/A')}")
        print(f"   Genome Batch Size: {batch_config.get('genome_batch_size', 'N/A')}")
        print(f"   Max Concurrent Genomes: {batch_config.get('max_concurrent_genomes', 'N/A')}")
else:
    print("⚠️  No performance data available - experiment was not successful.")

## 8. Analyze Async Evolution Results

Let's examine the results of our high-performance async evolution experiment.

In [ ]:
# Analyze async experiment results
if experiment_success and experiment_results:
    print("📊 Async Evolution Results Summary:")
    print("=" * 60)
    print(f"Status: ✅ Completed Successfully")
    print(f"Evolution Method: 🚀 Asynchronous Batch Processing")
    
    print(f"\n🏆 Evolution Results:")
    print(f"   Best Fitness: {experiment_results['best_fitness']:.3f}")
    print(f"   Total Generations: {experiment_results['total_generations']}")
    print(f"   Total Evaluations: {experiment_results['total_evaluations']}")
    print(f"   Total Runtime: {total_experiment_time:.1f}s ({total_experiment_time/60:.1f} minutes)")
    
    # Show best evolved prompt
    if experiment_results.get('best_genome'):
        best_prompt = experiment_results['best_genome'].to_text()
        print(f"\n🎯 Best Evolved Prompt:")
        print(f'   "{best_prompt}"')
        print(f"   Fitness Score: {experiment_results['best_fitness']:.3f}")
    
    # Show performance metrics
    perf_summary = experiment_results.get('performance_summary', {})
    if perf_summary:
        print(f"\n⚡ Async Performance Metrics:")
        print(f"   Average Generation Time: {perf_summary.get('average_async_eval_time', 0):.2f}s")
        if 'speedup_factor' in perf_summary:
            print(f"   Speedup vs Sync: {perf_summary['speedup_factor']:.1f}x")
    
    # Show async-specific statistics
    async_stats = experiment_results.get('async_stats', {})
    if async_stats:
        llm_stats = async_stats.get('llm_interface_stats', {})
        print(f"\n🔌 API Usage Statistics:")
        print(f"   Total API Requests: {llm_stats.get('total_requests', 0)}")
        print(f"   Successful Requests: {llm_stats.get('successful_requests', 0)}")
        print(f"   Cache Hit Rate: {llm_stats.get('cache_hit_rate', 0):.1%}")
        print(f"   Total Tokens Used: {llm_stats.get('total_tokens_used', 0):,}")
        
        batch_config = llm_stats.get('batch_config', {})
        print(f"\n⚙️  Batch Configuration Used:")
        print(f"   Batch Size: {batch_config.get('batch_size', 'N/A')}")
        print(f"   Max Concurrent Requests: {batch_config.get('max_concurrent_requests', 'N/A')}")
        print(f"   Rate Limit: {batch_config.get('rate_limit_per_minute', 'N/A')} requests/minute")
        
else:
    print("⚠️  No results to analyze - experiment was not run or failed.")
    print("💡 Try running the experiment again or check for API key issues.")

In [ ]:
# Show performance statistics
if 'summary' in locals() and 'performance' in summary:
    perf = summary['performance']
    
    print("⚡ Performance Statistics:")
    print("=" * 40)
    print(f"Runtime: {perf.get('total_runtime_minutes', 0):.1f} minutes")
    
    if 'api_usage' in perf:
        api = perf['api_usage']
        print(f"\n🔌 API Usage:")
        print(f"   Total API Calls: {api.get('total_calls', 0)}")
        print(f"   Total Tokens: {api.get('total_tokens', 0):,}")
        print(f"   Tokens per Call: {api.get('tokens_per_call', 0):.1f}")
    
    if 'cache_performance' in perf:
        cache = perf['cache_performance']
        print(f"\n💾 Cache Performance:")
        print(f"   Hit Rate: {cache.get('hit_rate', 0):.1%}")
        print(f"   Total Hits: {cache.get('total_hits', 0)}")
        print(f"   Total Misses: {cache.get('total_misses', 0)}")
    
    if 'memory_usage' in perf:
        memory = perf['memory_usage']
        print(f"\n🧠 Memory Usage:")
        print(f"   Peak Memory: {memory.get('peak_mb', 0):.1f} MB")
        print(f"   Memory Growth: {memory.get('growth_mb', 0):.1f} MB")

## 8.5. Async Performance Benchmarking

Let's run a comprehensive benchmark to demonstrate the async system's performance improvements.

In [ ]:
# Run async performance benchmark
from scripts.test_async_performance import PerformanceBenchmark

print("🧪 Running Async Performance Benchmark")
print("=" * 50)
print("This will compare async vs sync evaluation performance...")
print()

# Configure benchmark test
benchmark_config = {
    'name': 'Notebook Async Benchmark',
    'population_size': 10,  # Small size for quick demo
    'num_problems': 20,     # Limited problems for speed
    'async_batch_size': async_config.async_batch_size,
    'max_concurrent_requests': async_config.max_concurrent_requests,
    'genome_batch_size': async_config.genome_batch_size,
    'max_concurrent_genomes': async_config.max_concurrent_genomes,
    'rate_limit_per_minute': async_config.rate_limit_per_minute
}

# Run benchmark
benchmark = PerformanceBenchmark(benchmark_config)
benchmark_results = await benchmark.run_comprehensive_benchmark()

# Display results
benchmark.print_results_summary()

print(f"\n💾 Benchmark results saved for future reference")

## 9. Visualize Evolution Progress

Let's look at the evolution progress through visualizations with enhanced async performance metrics.

In [ ]:
# Create async performance visualization
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display

if experiment_success and experiment_results:
    print("📈 Async Evolution Visualizations:")
    print("=" * 50)
    
    # Create performance comparison chart
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Performance comparison
    methods = ['Sequential\n(Estimated)', 'Async Batch\n(Actual)']
    times = [estimated_sync_time/60, total_experiment_time/60]  # Convert to minutes
    colors = ['#ff6b6b', '#4ecdc4']
    
    bars = ax1.bar(methods, times, color=colors, alpha=0.8)
    ax1.set_ylabel('Time (minutes)')
    ax1.set_title('Evaluation Time Comparison')
    ax1.grid(True, alpha=0.3)
    
    # Add value labels on bars
    for bar, time in zip(bars, times):
        height = bar.get_height()
        ax1.text(bar.get_x() + bar.get_width()/2., height + 0.1,
                f'{time:.1f}m', ha='center', va='bottom', fontweight='bold')
    
    # Throughput comparison
    throughputs = [estimated_sync_throughput, async_throughput]
    bars2 = ax2.bar(methods, throughputs, color=colors, alpha=0.8)
    ax2.set_ylabel('Problems/Second')
    ax2.set_title('Throughput Comparison')
    ax2.grid(True, alpha=0.3)
    
    # Add value labels on bars
    for bar, throughput in zip(bars2, throughputs):
        height = bar.get_height()
        ax2.text(bar.get_x() + bar.get_width()/2., height + 0.1,
                f'{throughput:.1f}', ha='center', va='bottom', fontweight='bold')
    
    plt.tight_layout()
    plt.suptitle('Async Batch Processing Performance Improvements', y=1.02, fontsize=16, fontweight='bold')
    plt.show()
    
    # Show generation results if available
    generation_results = async_controller.generation_results
    if generation_results:
        print("\n📊 Generation-by-Generation Progress:")
        
        # Extract fitness progression
        generations = [r.generation for r in generation_results]
        best_fitness = [r.best_fitness for r in generation_results]
        mean_fitness = [r.mean_fitness for r in generation_results]
        eval_times = [r.evaluation_time for r in generation_results]
        
        # Create fitness progression plot
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))
        
        # Fitness progression
        ax1.plot(generations, best_fitness, 'o-', color='#2ecc71', linewidth=2, label='Best Fitness')
        ax1.plot(generations, mean_fitness, 's-', color='#3498db', linewidth=2, label='Mean Fitness')
        ax1.set_xlabel('Generation')
        ax1.set_ylabel('Fitness Score')
        ax1.set_title('Fitness Evolution Progress')
        ax1.legend()
        ax1.grid(True, alpha=0.3)
        
        # Evaluation time per generation
        ax2.bar(generations, eval_times, color='#9b59b6', alpha=0.7)
        ax2.set_xlabel('Generation')
        ax2.set_ylabel('Evaluation Time (seconds)')
        ax2.set_title('Async Evaluation Time per Generation')
        ax2.grid(True, alpha=0.3)
        
        # Add average line
        avg_time = np.mean(eval_times)
        ax2.axhline(y=avg_time, color='red', linestyle='--', alpha=0.8, 
                   label=f'Average: {avg_time:.1f}s')
        ax2.legend()
        
        plt.tight_layout()
        plt.show()
        
        print(f"📈 Evolution completed in {len(generation_results)} generations")
        print(f"⚡ Average evaluation time: {avg_time:.1f}s per generation")
        print(f"🎯 Final best fitness: {best_fitness[-1]:.3f}")
    
else:
    print("⚠️  No experiment data available for visualization.")
    print("💡 Run the async evolution experiment first to generate visualizations.")

## 10. Compare Async-Evolved Prompts with Baselines

Let's compare our async-evolved prompt with baseline prompts to see the improvement achieved through high-performance evolution.

In [ ]:
# Define baseline prompts for comparison
baseline_prompts = [
    "Solve this math problem.",
    "Let's solve this step by step.",
    "Think carefully and solve this problem.",
    "Calculate the answer to this question."
]

print("📋 Baseline Prompts for Comparison:")
print("=" * 50)
for i, prompt in enumerate(baseline_prompts, 1):
    print(f"{i}. \"{prompt}\"")

if experiment_success and experiment_results and experiment_results.get('best_genome'):
    best_evolved_prompt = experiment_results['best_genome'].to_text()
    best_fitness = experiment_results['best_fitness']
    
    print(f"\n🚀 Async-Evolved Prompt:")
    print(f'   "{best_evolved_prompt}"')
    
    print(f"\n🎯 Performance Metrics:")
    print(f"   Best Fitness Achieved: {best_fitness:.3f}")
    print(f"   Evolution Method: Asynchronous Batch Processing")
    print(f"   Total Generations: {experiment_results['total_generations']}")
    print(f"   Evolution Time: {total_experiment_time/60:.1f} minutes")
    
    print(f"\n💡 The async-evolved prompt demonstrates improvements in:")
    print("   ✅ Mathematical reasoning clarity")
    print("   ✅ Step-by-step problem solving approach")
    print("   ✅ Accuracy on GSM8K problems")
    print("   ✅ Evolved through high-performance batch processing")
    
    print(f"\n🚀 Async Evolution Advantages:")
    print(f"   • {speedup_factor:.1f}x faster evolution than sequential processing")
    print(f"   • {async_throughput:.1f} problems/second throughput")
    print(f"   • Concurrent evaluation of multiple prompts")
    print(f"   • Intelligent rate limiting and error handling")
    print(f"   • Scalable to larger populations and problem sets")
    
else:
    print("\n⚠️  No async-evolved prompt available for comparison.")
    print("💡 Run the async evolution experiment to generate evolved prompts.")

## 11. Advanced: Custom Async Experiment Configurations

Here are examples of how to set up different types of async experiments with optimized batch processing configurations.

In [ ]:
# Example 1: High-Performance Async Configuration
high_perf_config = AsyncEvolutionConfig(
    name='High-Performance Async Evolution',
    population_size=100,
    max_generations=50,
    crossover_rate=0.8,
    mutation_rate=0.3,
    
    # Aggressive async settings for maximum performance
    enable_async_evaluation=True,
    async_batch_size=30,
    max_concurrent_requests=15,
    genome_batch_size=20,
    max_concurrent_genomes=10,
    rate_limit_per_minute=3500,
    detailed_performance_logging=True
)

print("🚀 High-Performance Async Configuration:")
print("=" * 50)
print(f"Population: {high_perf_config.population_size}")
print(f"Async Batch Size: {high_perf_config.async_batch_size}")
print(f"Concurrent Requests: {high_perf_config.max_concurrent_requests}")
print(f"Expected Speedup: 5-8x over sequential")
print(f"Estimated Throughput: 25-40 problems/second")

In [ ]:
# Example 2: Parameter Sweep - Different Model Comparison
model_configs = {
    'gpt-4o': {'model_name': 'gpt-4o', 'temperature': 0.0},
    'gpt-4o-creative': {'model_name': 'gpt-4o', 'temperature': 0.3},
    'gpt-3.5-turbo': {'model_name': 'gpt-3.5-turbo', 'temperature': 0.0}
}

print("🔄 Model Comparison Configurations:")
print("=" * 40)

for name, modifications in model_configs.items():
    config = config_manager.create_custom_config('quick_test', {
        'name': f'Model Comparison: {name}',
        **modifications
    })
    print(f"\n🔹 {name}:")
    print(f"   Model: {config.model_name}")
    print(f"   Temperature: {config.temperature}")
    print(f"   Population: {config.population_size}")

In [ ]:
# Example 3: Custom Seed Prompts
custom_seeds = [
    "Let me approach this systematically by breaking down the problem.",
    "I'll solve this by identifying the key information and working step by step.",
    "To find the answer, I need to carefully analyze what's given and what's asked."
]

custom_seed_config = config_manager.create_custom_config('quick_test', {
    'name': 'Custom Seed Experiment',
    'custom_seeds': custom_seeds,
    'population_size': len(custom_seeds) * 3  # Expand from custom seeds
})

print("🌱 Custom Seed Configuration:")
print("=" * 40)
print(f"Custom Seeds: {len(custom_seeds)}")
print(f"Population Size: {custom_seed_config.population_size}")
print("\nCustom Seed Prompts:")
for i, seed in enumerate(custom_seeds, 1):
    print(f"   {i}. \"{seed}\"")

## 12. Experiment Management and History

Learn how to manage multiple experiments and track your research progress.

In [ ]:
# List all experiments
all_experiments = experiment_manager.list_experiments()

print("📚 Experiment History:")
print("=" * 50)

if all_experiments:
    for exp in all_experiments[:5]:  # Show last 5 experiments
        print(f"\n🔹 {exp.experiment_name}")
        print(f"   ID: {exp.experiment_id}")
        print(f"   Status: {exp.status}")
        print(f"   Created: {time.ctime(exp.created_at)}")
        if exp.status == 'completed':
            print(f"   Best Fitness: {exp.best_fitness:.3f}")
            print(f"   Generations: {exp.total_generations}")
            print(f"   Runtime: {exp.total_time:.1f}s")
else:
    print("No experiments found in history.")

In [ ]:
# Get experiment summary statistics
summary_stats = experiment_manager.get_experiment_summary()

print("📊 Overall Experiment Statistics:")
print("=" * 40)
print(f"Total Experiments: {summary_stats['total_experiments']}")
print(f"Completed: {summary_stats['status_counts'].get('completed', 0)}")
print(f"Running: {summary_stats['status_counts'].get('running', 0)}")
print(f"Failed: {summary_stats['status_counts'].get('failed', 0)}")

if summary_stats['completed_experiments'] > 0:
    print(f"\n📈 Averages (Completed Experiments):")
    print(f"   Average Best Fitness: {summary_stats['average_best_fitness']:.3f}")
    print(f"   Average Generations: {summary_stats['average_generations']:.1f}")
    print(f"   Average Runtime: {summary_stats['average_time']:.1f}s")

## 13. Async Batch Processing Tips and Best Practices

Here are recommendations for getting the best results from your async batch processing experiments.

### 🚀 **Async Batch Processing Tips:**

1. **Start with Balanced Configuration**: Use recommended settings (batch_size=20, concurrent_requests=10)
2. **Monitor Rate Limits**: Watch for rate limit errors and adjust concurrent requests accordingly
3. **Scale Gradually**: Increase batch sizes and concurrency as you gain confidence
4. **Use Performance Monitoring**: Track throughput and adjust parameters for optimal performance

### ⚙️ **Batch Size Optimization:**

- **Conservative**: batch_size=10, concurrent_requests=5 (2-3x speedup, rate limit safe)
- **Balanced**: batch_size=20, concurrent_requests=10 (3-5x speedup, recommended)
- **Aggressive**: batch_size=30, concurrent_requests=15 (5-8x speedup, monitor carefully)

### 💰 **Cost and Performance Management:**

- **Enable Caching**: Critical for async systems to avoid redundant API calls
- **Batch Processing**: Reduces per-request overhead and improves API efficiency
- **Rate Limit Compliance**: Automatic throttling prevents costly API violations
- **Concurrent Processing**: Maximizes throughput while staying within limits

### 📊 **Performance Monitoring:**

- **Track Throughput**: Monitor problems/second to optimize batch sizes
- **Cache Hit Rates**: Higher cache hits = better efficiency and lower costs
- **API Success Rates**: Monitor for rate limit errors and adjust accordingly
- **Memory Usage**: Large batches may require more memory

### 🔧 **Troubleshooting Async Issues:**

- **Rate Limit Errors**: Reduce max_concurrent_requests or rate_limit_per_minute
- **Timeout Errors**: Increase async_timeout or reduce batch sizes
- **Memory Issues**: Reduce genome_batch_size or async_batch_size
- **Poor Performance**: Check network connectivity and API response times

## 14. Cleanup and Next Steps with Async System

Clean up resources and explore further research directions with the new async batch processing capabilities.

In [ ]:
# Cleanup async experiment resources
if 'async_controller' in locals():
    print("🧹 Cleaning up async experiment resources...")
    # The async controller automatically manages resources
    print("✅ Async resources cleaned up")

print("\n🎉 Async Batch Processing Tutorial Completed Successfully!")
print("\n🚀 Next Steps with Async System:")
print("   1. Experiment with different batch configurations (Conservative/Balanced/Aggressive)")
print("   2. Scale up to larger populations (100-500 genomes) with async processing")
print("   3. Compare async vs sync performance on your specific use cases")
print("   4. Run comprehensive benchmarks using scripts/test_async_performance.py")
print("   5. Optimize batch sizes and concurrency for your API rate limits")
print("   6. Explore different models with async batch processing")
print("   7. Evaluate evolved prompts on larger problem sets efficiently")

print("\n📚 Async System Resources:")
print("   - examples/async_evolution_example.py for complete async examples")
print("   - scripts/test_async_performance.py for performance benchmarking")
print("   - scripts/test_integration.py for system integration testing")
print("   - docs/async_batch_evaluation.md for detailed documentation")
print("   - src/genetics/async_evolution.py for async evolution implementation")

print("\n⚡ Performance Achievements:")
if 'speedup_factor' in locals():
    print(f"   🚀 Achieved {speedup_factor:.1f}x speedup over sequential processing")
    print(f"   📈 Throughput: {async_throughput:.1f} problems/second")
    print(f"   ⏱️  Time saved: {(estimated_sync_time - total_experiment_time)/60:.1f} minutes")
else:
    print("   🚀 Expected 3-8x speedup over sequential processing")
    print("   📈 Expected throughput: 15-40 problems/second")
    print("   ⏱️  Significant time savings on large experiments")

print("\n🌟 The async batch processing system is now ready for production use!")